In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [4]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

In [5]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [6]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [7]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [8]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [9]:
compras = (pd.concat
                ([
                pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'),
                productos_sin_compra
               ])
        )


cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras[cols_to_Int64] = compras[cols_to_Int64].astype('Int64')

compras['tolerance_order_date'] = compras['order_date'].dt.normalize() - pd.Timedelta(days=5)

In [10]:
costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        # tolerance = pd.Timedelta(days=1),
        
        direction = 'backward')
)

# Checks

In [11]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Todo correcto con check1


In [12]:
check2 = costo_venta[costo_venta['product_price'].isna()][['product_id', 'product_name_x']]
print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 21 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
10061,14757,Nuevo *0*
10272,28586,Anticipo
13415,28638,Servicios de Facturación
18005,4,Anticipo (PdV)


# Pruebas

In [13]:
product_id_to_check = 13811

In [14]:
prod_costo_venta_gen = (costo_venta
    .loc[costo_venta['product_id'] == product_id_to_check]
        [['name', 'invoice_date', 'salesperson_name', 'quantity',
        'price_subtotal', 'product_price', 'order_id', 'order_name',]]
)

prod_compras = compras.loc[(compras['product_id_pp'] == product_id_to_check)]


# writer = pd.ExcelWriter('comparativa_costo_venta.xlsx', engine="openpyxl")

# prod_costo_venta_gen.to_excel(writer, sheet_name='costo_venta')
# prod_compras.to_excel(writer, sheet_name='compras')

# writer.close()
# writer.handles = None

In [15]:
compras['product_qty'] = compras['product_qty'].astype('Float64')
ventas_año['salesperson_name'] = ventas_año['salesperson_name'].astype('object')

In [16]:
wep_costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('order_date'), 
        
        left_by = ['product_id', 'salesperson_name', 'quantity'], 
        right_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        
        left_on = 'invoice_date', 
        right_on = 'order_date', 

        tolerance = pd.Timedelta(days=5),
        
        direction = 'nearest')
)


wep_prod_costo_venta_gen = (wep_costo_venta
    .loc[wep_costo_venta['product_id'] == product_id_to_check]
        [['name', 'invoice_date', 'salesperson_name', 'quantity',
        'price_subtotal', 'product_price', 'order_id', 'order_name',]]
)


# writer = pd.ExcelWriter('wep_comparativa_costo_venta.xlsx', engine="openpyxl")

# wep_prod_costo_venta_gen.to_excel(writer, sheet_name='wep_costo_venta')
# prod_compras.to_excel(writer, sheet_name='compras')

# writer.close()
# writer.handles = None

In [17]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [18]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]

In [19]:
prod_varios_prov = compras[['partner_id', 'partner_name', 'product_id_pp', 'product_name']].sort_values('partner_id').groupby('product_id_pp').agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
prod_varios_prov.columns = ['product_name', 'mean', 'first']
prod_varios_prov['diff'] = prod_varios_prov['mean']  == prod_varios_prov['first']
prod_varios_prov.loc[prod_varios_prov['diff'] == False]

,product_name,mean,first,diff
product_id_pp,,,,
7480,"[2538] Rondana Plana 1/4"" C#181 *2538*","5,337.00",5221,False
7481,[2540] Rondana Plana 3/16 C#184 *2540*,"5,293.50",5221,False
7483,"[6528] Pija Autorr. C/Rondana 8""*1/2"" Galv. *6...","5,345.29",5221,False
7510,"[5388] Tuerca Hexagonal 1/4"" Hncz (5600/Ct) C...","5,329.75",5221,False
7511,"[10179] Taq. Plastico El-Pro 1/4"" Blanco TQ-01...","5,328.20",5313,False
...,...,...,...,...
29236,[4413] Brida Flexible Coflex PB-300 Larga *4413*,"5,326.33",5309,False
29241,[4426] Color Cemento Negro *4426*,"5,236.50",5143,False
29255,[4497] Guantes De Corte De Nitrilo 48-22-8902 ...,"5,179.80",5128,False


In [20]:
prod_varios_prov_ids = prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp']
prod_varios_prov_ids

0       7480
1       7481
2       7483
3       7510
4       7511
       ...  
206    29236
207    29241
208    29255
209    29436
210    29452
Name: product_id_pp, Length: 211, dtype: Int64

In [25]:
ven_prod_diversos = ventas_año[ventas_año['product_id'].isin(prod_varios_prov_ids)]
ven_prod_diversos

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,warehouse,fact_line_id,product_id,product_name,prod_codigo,prod_linea,quantity,price_unit,discount,price_subtotal
9,125,F1-CC/2024/00004,2024-01-02,posted,Shop/0007,PdV,17,out_invoice,<NA>,<NA>,...,A1,345,8002,"Tuerca Hexagonal 3/4"" Hncz (360/Ct) C#2 *5393*",5393,000CE,2.00,3.60,0.00,7.20
53,190,F1-CC/2024/00012,2024-01-02,posted,Shop/0025,PdV,57,out_invoice,<NA>,<NA>,...,A1,579,11326,"Pija Drywall 8 x 2"" 15422 (Kg) *15112*",15112,102,1.00,105.95,0.00,105.95
65,226,F2-CC/2024/00023,2024-01-02,posted,PdV SJC/0005,PdV,14,out_invoice,<NA>,<NA>,...,A2,738,7511,"Taq. Plastico El-Pro 1/4"" Blanco TQ-01 *10179*",10179,45,100.00,0.46,0.00,46.00
69,236,F2-CC/2024/00025,2024-01-02,posted,PdV SJC/0030,PdV,61,out_invoice,<NA>,<NA>,...,A2,778,9169,Bis. Bidi. Brazo Corto #3655 (Par) *13374*,13374,00CHA,2.00,29.79,0.00,59.58
79,252,F1-CC/2024/00013,2024-01-02,posted,Shop/0031,PdV,70,out_invoice,<NA>,<NA>,...,A1,837,7480,"Rondana Plana 1/4"" C#181 *2538*",2538,126,50.00,0.34,0.00,17.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62613,107903,F2-CC/2024/12056,2024-06-24,posted,PdV SJC/12266,PdV,22354,out_invoice,<NA>,<NA>,...,A2,405006,9169,Bis. Bidi. Brazo Corto #3655 (Par) *13374*,13374,00CHA,5.00,29.79,0.00,148.95
62627,107911,F2-CC/2024/12057,2024-06-24,posted,PdV SJC/12267,PdV,22358,out_invoice,<NA>,<NA>,...,A2,405040,10213,Sil.Pens Acrilastic Blanco 300ml *9537*,9537,00PEN,6.00,59.31,0.00,355.86
62644,107936,F2-CC/2024/12059,2024-06-24,posted,PdV SJC/12269,PdV,22365,out_invoice,<NA>,<NA>,...,A2,405123,10213,Sil.Pens Acrilastic Blanco 300ml *9537*,9537,00PEN,1.00,59.31,0.00,59.31
62715,108033,F2-CC/2024/12071,2024-06-24,posted,PdV SJC/12281,PdV,22389,out_invoice,<NA>,<NA>,...,A2,405467,7480,"Rondana Plana 1/4"" C#181 *2538*",2538,126,12.00,0.34,0.00,4.08


In [21]:
compras[compras['product_id_pp'].isin(prod_varios_prov_ids)]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
2,19106,3138,P03124 (C 175301),2024-06-27 00:45:34,5280,Distribuidora De Herrajes Y Aluminio De Los Cabos,13833,[25982] Bisagra Vidrio-Vidrio #1432 Herralum *...,10.00,253.80,False,C 175301,2024-06-26,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores,2024-06-22
13,19096,3132,P03118 (FA 050191),2024-06-26 23:46:59,5250,El Fluxometro De Cabos,8759,[44394] Cubre Tornillos *44394*,10.00,9.04,False,FA 050191,2024-06-26,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-21
14,19092,3131,P03117 (SJCFCO 16621),2024-06-26 23:42:17,5326,Madereria El Pino De Los Cabos,13943,[15143] Triplay Birch 1/2 4'*8' *15143*,2.00,500.00,False,SJCFCO 16621,2024-06-26,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-21
15,19093,3131,P03117 (SJCFCO 16621),2024-06-26 23:42:17,5326,Madereria El Pino De Los Cabos,14075,[40032] Triplay Birch 5/8 4'*8' *40032*,3.00,616.38,False,SJCFCO 16621,2024-06-26,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-21
16,19094,3131,P03117 (SJCFCO 16621),2024-06-26 23:42:17,5326,Madereria El Pino De Los Cabos,13331,[16320] Triplay Birch 1/4 4'*8' *16320*,2.00,267.24,False,SJCFCO 16621,2024-06-26,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,<NA>,<NA>,SAE,2023-10-16 00:00:00,<NA>,NaN,14016,Maletin D/Broca SDS-Plus MK D-31734 13pz *8635*,<NA>,599.00,NaN,NaN,NaT,NaN,NaN,2023-10-11
6085,<NA>,<NA>,SAE,2023-11-23 00:00:00,<NA>,NaN,14671,Malla Sombra 2x100m 90% Beige Raschel *96679*,<NA>,"3,620.70",NaN,NaN,NaT,NaN,NaN,2023-11-18
6768,<NA>,<NA>,SAE,2023-01-19 00:00:00,<NA>,NaN,27422,Silicon Cafe DAP Alex Plus 300 ml *2468*,<NA>,82.76,NaN,NaN,NaT,NaN,NaN,2023-01-14
7262,<NA>,<NA>,SAE,2023-11-08 00:00:00,<NA>,NaN,28253,Detector Tipo Lapiz 1AC-A1-II Mca Fluke *11508*,<NA>,828.25,NaN,NaN,NaT,NaN,NaN,2023-11-03


In [31]:
ven_prod_diversos

costo_venta_prod_diversos = (
    pd.merge_asof(
        left = ven_prod_diversos.sort_values('invoice_date'),
        right = compras.sort_values('order_date'), 
        
        left_by = ['product_id', 'salesperson_name', 'quantity'], 
        right_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        
        left_on = 'invoice_date', 
        right_on = 'order_date', 

        tolerance = pd.Timedelta(days=5),
        
        direction = 'nearest')
)

costo_venta_prod_diversos

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,product_id_pp,product_name_y,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
0,125,F1-CC/2024/00004,2024-01-02,posted,Shop/0007,PdV,17,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
1,428,F2-CC/2024/00062,2024-01-02,posted,PdV SJC/0050,PdV,96,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
2,403,F2-CC/2024/00055,2024-01-02,posted,PdV SJC/0058,PdV,104,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
3,349,F1-CC/2024/00025,2024-01-02,posted,Shop/0045,PdV,112,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
4,299,F2-CC/2024/00035,2024-01-02,posted,PdV SJC/0049,PdV,94,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3959,107736,F2-CC/2024/12039,2024-06-24,posted,PdV SJC/12249,PdV,22312,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
3960,107698,F1-CC/2024/09635,2024-06-24,posted,PdV CSL/9611,PdV,22300,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
3961,108033,F2-CC/2024/12071,2024-06-24,posted,PdV SJC/12281,PdV,22389,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT
3962,107750,F1-CC/2024/09644,2024-06-24,posted,PdV CSL/9620,PdV,22316,out_invoice,<NA>,<NA>,...,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaT,NaN,NaN,NaT


In [30]:
costo_venta_prod_diversos[~costo_venta_prod_diversos['product_price'].isna()]

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,product_id_pp,product_name_y,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
13,818,F2-VS/2024/00003,2024-01-03,posted,S00110,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14272,[13508] Tablon Americano 2x12x12 *13508*,2.00,910.34,False,SJCFCO 68,NaT,<NA>,Yamilet Blanco Salas,2023-12-29
16,722,F2-VS/2024/00002,2024-01-03,posted,S00160,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14147,[3637] Triplay Pino 3/4 4'*8' 1/Cara *3637*,20.00,646.55,False,SJCFCO 68,NaT,<NA>,Yamilet Blanco Salas,2023-12-29
46,1552,F1-VS/2024/00024,2024-01-05,posted,S00365,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14147,[3637] Triplay Pino 3/4 4'*8' 1/Cara *3637*,2.00,646.55,False,MPIFCO 125,NaT,<NA>,Mayra Angelica Parada Manjarrez,2023-12-30
47,1553,F1-VS/2024/00025,2024-01-05,posted,S00399,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14772,[16313] Pegamento Titebond Verde1L *16313*,6.00,245.69,False,MPIFCO 125,NaT,<NA>,Mayra Angelica Parada Manjarrez,2023-12-30
57,1546,F1-VS/2024/00023,2024-01-05,posted,S00084,Ventas,<NA>,out_invoice,<NA>,<NA>,...,13636,[13756] Tabla Americana 1x12x10 *13756*,5.00,356.03,False,A 22053,NaT,<NA>,Mayra Angelica Parada Manjarrez,2023-12-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3906,107082,F2-VS/2024/01510,2024-06-21,posted,S27756,Ventas,<NA>,out_invoice,<NA>,<NA>,...,28609,[1961] Poste Metálico Perfirey CAL.26 4.10 X 3...,6.00,70.44,False,LC 9702,2024-06-21,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-17
3907,107082,F2-VS/2024/01510,2024-06-21,posted,S27756,Ventas,<NA>,out_invoice,<NA>,<NA>,...,11538,[29922] Fibracinta 4x150 65 g *29922*,1.00,84.69,False,LC 9702,2024-06-21,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-17
3916,106284,F1-VS/2024/01842,2024-06-21,posted,S29357,Ventas,<NA>,out_invoice,<NA>,<NA>,...,28395,"[12672] Durock 1/2"" 4x8 *12672*",4.00,820.00,False,55879,2024-06-21,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores,2024-06-20
3920,106317,F2-VS/2024/01508,2024-06-21,posted,S29553,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14548,[24622] Tabla Teka Nacional 1x8 (Cama) *24622*,4.00,"1,820.64",False,SJCFCO 16134,2024-06-21,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-16
